In [20]:
!pip install --upgrade peft diffusers accelerate transformers

In [21]:
from diffusers import MotionAdapter
import torch

In [22]:
adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16)

The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.


In [23]:
from diffusers import AnimateDiffPipeline, DDIMScheduler
from diffusers.utils import export_to_gif

In [24]:
!pip install torchinfo

In [25]:
from torchinfo import summary

summary(adapter)

Layer (type:depth-idx)                                                      Param #
MotionAdapter                                                               --
├─MotionModules: 1-1                                                        --
│    └─ModuleList: 2-1                                                      --
│    │    └─TransformerTemporalModel: 3-1                                   36,071,680
├─ModuleList: 1-2                                                           --
│    └─MotionModules: 2-2                                                   --
│    │    └─ModuleList: 3-2                                                 4,519,040
│    └─MotionModules: 2-3                                                   --
│    │    └─ModuleList: 3-3                                                 18,049,280
│    └─MotionModules: 2-4                                                   --
│    │    └─ModuleList: 3-4                                                 72,143,360
│    └─MotionMod

In [26]:
model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
pipe = AnimateDiffPipeline.from_pretrained(model_id, motion_adapter=adapter, torch_dtype=torch.float16)
scheduler = DDIMScheduler.from_pretrained(
    model_id,
    subfolder="scheduler",
    clip_sample=False,
    timestep_spacing="linspace", # line space
    beta_schedule="linear",
    steps_offset=1,
)

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

The config attributes {'center_input_sample': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'mid_block_type': 'UNetMidBlock2DCrossAttn', 'only_cross_attention': False, 'attention_head_dim': 8, 'dual_cross_attention': False, 'class_embed_type': None, 'addition_embed_type': None, 'num_class_embeds': None, 'upcast_attention': False, 'resnet_time_scale_shift': 'default', 'resnet_skip_time_act': False, 'resnet_out_scale_factor': 1.0, 'time_embedding_type': 'positional', 'time_embedding_dim': None, 'time_embedding_act_fn': None, 'timestep_post_act': None, 'conv_in_kernel': 3, 'conv_out_kernel': 3, 'projection_class_embeddings_input_dim': None, 'class_embeddings_concat': False, 'mid_block_only_cross_attention': None, 'cross_attention_norm': None, 'addition_embed_type_num_heads': 64} were passed to UNetMotionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [27]:
pipe.scheduler = scheduler

# enable memory savings
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

output = pipe(
    prompt=(
        "masterpiece, best quality, highly detailed, ultra detailed, sunset, "
        "orange sky, warm lighting, fishing boats, ocean waves, seagulls, "
        "rippling water, wharf, silhouette, serene atmosphere, dusk, "
        "evening glow, golden hour, coastal landscape, seaside scenery"
    ),
    negative_prompt="bad quality, worse quality",
    num_inference_steps=25,
    num_images_per_prompt=16,
    guidance_scale=7.5,
    generator=torch.Generator("cpu").manual_seed(55),
)


  0%|          | 0/25 [00:00<?, ?it/s]

In [28]:
frames = output.frames[0]

In [29]:
export_to_gif(frames, "animation.gif")

'animation.gif'

In [30]:
import imageio
import requests
import torch
from diffusers import AnimateDiffVideoToVideoPipeline, DDIMScheduler, MotionAdapter
from diffusers.utils import export_to_gif
from io import BytesIO
from PIL import Image

In [31]:
# Load the motion adapter
adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16).to("cuda")
# Load SD 1.5 based finetuned model
model_id = "SG161222/Realistic_Vision_V5.1_noVAE"
pipe = AnimateDiffVideoToVideoPipeline.from_pretrained(model_id, motion_adapter=adapter, torch_dtype=torch.float16).to("cuda")
scheduler = DDIMScheduler.from_pretrained(
    model_id,
    subfolder="scheduler",
    clip_sample=False,
    timestep_spacing="linspace",
    beta_schedule="linear",
    steps_offset=1
)
pipe.scheduler = scheduler

# enable memory savings
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

# helper function to load videos
def load_video(file_path: str):
  images = []

  if file_path.startswith(('http://', 'https://')):
    # If the file_path is a URL
    response = requests.get(file_path)
    response.raise_for_status()
    content = BytesIO(response.content)
    vid = imageio.get_reader(content)
  else:
    # Assuming it's a local file path
    vid = imageio.get_reader(file_path)

  for frame in vid:
    pil_image = Image.fromarray(frame)
    images.append(pil_image)

  return images

video = load_video("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/animatediff-vid2vid-input-1.gif")

output = pipe(
    video = video,
    prompt="A robot playing a guitar, on a boat, in the ocean, high quality",
    negative_prompt="bad quality, worse quality",
    guidance_scale=5,
    num_inference_steps=100,
    strength=0.9,
    generator=torch.Generator("cuda").manual_seed(42),
)
frames = output.frames[0]
export_to_gif(frames, "animation2.gif")

The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

The config attributes {'center_input_sample': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'mid_block_type': 'UNetMidBlock2DCrossAttn', 'only_cross_attention': False, 'attention_head_dim': 8, 'dual_cross_attention': False, 'class_embed_type': None, 'addition_embed_type': None, 'num_class_embeds': None, 'upcast_attention': False, 'resnet_time_scale_shift': 'default', 'resnet_skip_time_act': False, 'resnet_out_scale_factor': 1.0, 'time_embedding_type': 'positional', 'time_embedding_dim': None, 'time_embedding_act_fn': None, 'timestep_post_act': None, 'conv_in_kernel': 3, 'conv_out_kernel': 3, 'projection_class_embeddings_input_dim': None, 'class_embeddings_concat': False, 'mid_block_only_cross_attention': None, 'cross_attention_norm': None, 'addition_embed_type_num_heads': 64} were passed to UNetMotionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


  0%|          | 0/90 [00:00<?, ?it/s]

'animation2.gif'